In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['./cnn_finetune'])

from load_cifar import load_cifar10_data, load_cifar100_data

import numpy as np; print('numpy:', np.__version__)
import sklearn; print('sklearn:', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux


Using TensorFlow backend.


numpy: 1.16.2
sklearn: 0.20.3
tensorflow 1.13.1
keras 2.2.4


In [2]:
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


config = tf.ConfigProto()

# cpu
# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores, allow_soft_placement=True, device_count={'CPU': 4})
    
# gpu allow_growth
config.gpu_options.allow_growth = True

# set memory limit of gpu
# config.gpu_options.per_process_gpu_memory_fraction = 0.5

set_session(tf.Session(config=config))

In [3]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

from cnn_finetune.darknet53 import darknet53_model

In [4]:
# Example to fine-tune on 3000 samples from Cifar10
model_name = 'darknet53'
dataset_name = 'cifar10'
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 10
batch_size = 32
epochs = 999
load_weights = True
fine_tuning = True

# Load Cifar10 data. Please implement your own load_data() module for your own dataset
if dataset_name == 'cifar10':
    X_train, Y_train, X_valid, Y_valid = load_cifar10_data(img_rows, img_cols)
elif dataset_name == 'cifar100':
    X_train, Y_train, X_valid, Y_valid = load_cifar100_data(img_rows, img_cols)
else:
    raise('no method for loading dataset')

# Load our model
model = darknet53_model(img_rows, img_cols, channel, num_classes, fine_tuning, load_weights)

cifar10 trainset: <class 'numpy.ndarray'> (50000, 32, 32, 3) <class 'numpy.ndarray'> (50000, 1)
cifar10 valset: <class 'numpy.ndarray'> (10000, 32, 32, 3) <class 'numpy.ndarray'> (10000, 1)
----------------------------------------------------------------------------------------------------
Instructions for updating:
Colocations handled automatically by placer.
darknet53_model: classes: 10 fine-tuning: True
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
_______________________________________________________________________________________________

In [ ]:
# Learning rate is changed to 0.001
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
sgd = SGD(lr=0.1, decay=5e-4, momentum=0.9, nesterov=True)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

checkpoint = ModelCheckpoint("./models/" + model_name + "/"+ dataset_name +"-e" + "{epoch:02d}-{val_loss:.4f}.hd5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=True, 
                             mode='auto', 
                             period=1)

checkpoint_best = ModelCheckpoint("./models/" + model_name + "/"+ dataset_name + "-best.hd5", 
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only=True, 
                                  save_weights_only=False,
                                  mode='auto', 
                                  period=1)

earlyStopping = EarlyStopping(monitor='val_loss', 
                              min_delta=0, 
                              patience=10, 
                              verbose=1, 
                              mode='auto', 
                              baseline=None, 
                              restore_best_weights=False)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=1e-6, cooldown=0, min_lr=0)

tensorBoard = TensorBoard(log_dir='./models/' + model_name +'/logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

csvLogger = CSVLogger(filename='./models/'+ model_name + '/' + dataset_name + "-log.csv", separator=',', append=False)


In [ ]:
# Start Fine-tuning
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          validation_data=(X_valid, Y_valid),
          callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
          )

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/999
50000/50000 [==============================] - 618s 12ms/step - loss: 3.1344 - acc: 0.6319 - val_loss: 2.9697 - val_acc: 0.3548

Epoch 00001: val_loss improved from inf to 2.96975, saving model to ./models/darknet53/cifar10-e01-2.9697.hd5

Epoch 00001: val_loss improved from inf to 2.96975, saving model to ./models/darknet53/cifar10-best.hd5
Epoch 2/999
21856/50000 [============>.................] - ETA: 4:30 - loss: 1.8417 - acc: 0.6659

In [ ]:
# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)
print(Y_valid.shape, predictions_valid.shape)

In [ ]:
print('cross-entropy:', sklearn.metrics.log_loss(Y_valid, predictions_valid))

y_true = np.argmax(Y_valid, axis=1)
y_hat = np.argmax(predictions_valid, axis=1)
# print('report:', sklearn.metrics.classification_report(y_true, y_hat))
print('acc:', sklearn.metrics.accuracy_score(y_true, y_hat))
# print('roc:', sklearn.metrics.roc_auc_score(y_true, y_hat))
# print('confusion_matrix:\n', sklearn.metrics.confusion_matrix(y_true, y_hat))